# PRIM CHARIF Mahdi

## Importing the libraries

In [1]:
from zipfile import ZipFile 
import json
import pandas as pd
import numpy as np
from annoy import AnnoyIndex
from sentence_transformers import SentenceTransformer,util
import torch
import os
from functools import reduce
from html.parser import HTMLParser
import codecs
import os
import urllib.request
import urllib.error
from urllib.request import urlopen
from bs4 import BeautifulSoup
import time
import requests
import csv
import json
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
import pickle
import math
import heapq
import re
import wikipedia

## Test with the Wikipedia library

In [5]:
model = SentenceTransformer('distiluse-base-multilingual-cased')

In [6]:
a = wikipedia.summary("Godfather (1991 film)")

In [7]:
b = wikipedia.summary("God Father (1995 film)")

In [8]:
c = wikipedia.summary("Inter Milan")

In [9]:
d = wikipedia.summary("Juventus")

In [10]:
e = wikipedia.summary("Oasis (music)")

In [11]:
L = [a,b,c,d,e]
corpus_embeddings = model.encode(L)

In [16]:
for query in L:
    query_embedding = model.encode(query, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    cos_scores = cos_scores.cpu()

    #We use torch.topk to find the highest 5 scores
    top_results = torch.topk(cos_scores, k = 2)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print(L[idx].split()[:2], "(Score: %.4f)" % (score))





Query: Le Parrain (The Godfather) est un film américain de Francis Ford Coppola, sorti en 1972.
Produit par les studios Paramount, le film est une adaptation du roman du même nom de Mario Puzo. L'histoire se déroule de 1945 à 1955 et est centrée sur les Corleone, une des plus grandes familles de la Mafia américaine. Le film aborde le sujet de la succession du patriarche de la famille, Vito Corleone dit le Parrain (Marlon Brando), et de l'ascension de son plus jeune fils, Michael (Al Pacino), qui initialement souhaite rester en dehors des activités criminelles de la famille. Mais, à cause d'un enchaînement de circonstances tragiques, Michael finit par en devenir le membre le plus impitoyable.
Le Parrain est considéré comme l'un des plus grands films du cinéma mondial et l'un des plus influents, spécialement dans le genre des films de gangsters. Il est classé à la deuxième place des meilleurs films du cinéma nord-américain par l'American Film Institute (AFI) (derrière Citizen Kane). 


Query: Oasis est un groupe de rock alternatif britannique, originaire de Manchester. Formé en 1991, et initialement nommé The Rain, le groupe est au départ composé de Liam Gallagher (chant), Paul « Bonehead » Arthurs (guitare), Paul « Guigsy » McGuigan (basse) et Tony McCarroll (batterie), rapidement rejoint par Noel (guitare principale et chant), le frère aîné de Liam.
Oasis signe avec le label indépendant Creation Records en 1993 et sort son premier album, Definitely Maybe, en 1994. L'année suivante, alors qu'une rivalité naît avec le groupe Blur, le groupe enregistre (What's the Story) Morning Glory? avec le batteur Alan White. Ce nouvel album devient l'un des albums les plus vendus de tous les temps, avec plus de 22 millions d'exemplaires vendus dans le monde entier, et les frères Gallagher apparaissent régulièrement dans les tabloïds pour leurs disputes fraternelles et leur mode de vie sauvage. En 1996, Oasis se produit deux fois à Knebworth devant un public de 125 000 personnes 

# ----------------------------

Problem : I must provide a precise query to the Wikipedia library, otherwise I would get an error.
I will download the pages and preprocess the HTML find I will get.

## Downloading the data

In [47]:
link = open("C:/Users/chari/PRIM/movies.html", 'r')

In [48]:
class MyHTMLParser(HTMLParser):
    def __init__(self):
        self.d = []
        self.info = [] # This list will be useful to get the info from the infobox of wikipedia
        self.tag = [] # We will use that to know when we have to stop retrieving the paragraphs
        super().__init__()

    def handle_starttag(self, tag, attrs):
        self.tag = []
        self.tag.append(tag)
        return(tag)
        
    def handle_data(self, data):
        if data.startswith('https'):
            self.d.append(data)
        else:
            self.info.append(data)
        return(data)

    def return_data(self):
        return(self.d)
    
    def return_info(self):
        return(self.info)
    
    def return_tag(self):
        if len(self.tag)!= 0:
            return(self.tag[0])
    
# To check if it is an url, we check if it begins with "https"

In [49]:
urls = []

parser = MyHTMLParser()
for item in list(link):
    parser.feed(item)
    urls += parser.return_data()
parser.close()

In [50]:
len(urls)

10000

In [55]:
for url in urls:
    try:
        name = 'C:/Users/chari/PRIM/moviesfolder/'+ url.split("/")[-1] + '.html'
        urllib.request.urlretrieve(url, name)
    except urllib.error.HTTPError as err:
        print(err.code)
    except Exception :
        print("error")

error
404
404
404


## Preprocessing the data

In [107]:
for link_movie in os.listdir("moviesfolder"):
    tot = []
    A = []
    B = []
    
    try:
        file = open("moviesfolder/" + link_movie, encoding='utf8')
        soup = BeautifulSoup(file, 'html.parser')
        
        # We have to check if the page refers to a film or is a disambiguous page. We do not want to save the disambiguous ones.
        disambiguous = soup.find("table", {"id" : "disambigbox"})
        
        if disambiguous is not None :
            continue
            
        # We get the title of the film

        # We put a condition to ONLY retrieve the title of the film, without the parenthesis containing the word "movie" and the year

        if soup.select_one('#firstHeading').find("i") != None:

            title = soup.select_one('#firstHeading').find("i").text

        else:

            title  = soup.select_one('#firstHeading').text   

        tot.append(title)

        # We get the intro and the plot (1st and 2nd sections)

        intro = ''
        plot = ''


        # We get the intro first

        tag  = soup.select_one('p')

        if tag != None:

            if tag.find_parent('table') == None and tag.text!="\n": # We get the intro if it is not a simple "\n".
                while tag.name == 'p':
                    intro+=tag.text
                    tag = tag.find_next_sibling()


            else: # If the paragraph is in the infobox, we don't append the text but look for the next section

                while tag.text == "\n" or tag.find_parent('table') != None :
                    tag = tag.find_next("p")


                while tag.name == 'p': # While we encounter <p> tags, we add them to the intro. Once a different tag is found, it would mean that we reached the end of the intro
                    intro+=tag.text
                    tag = tag.find_next_sibling()


            # We can now retrieve the second section

            tag = tag.find_next("p") # Now that we left the loop, we are looking for the first p, which would mean we entered the second section

            if tag!= None:

                if tag.find_parent('table') == None and tag.next!="\n" and tag.find_parent('blockquote') == None: # We added a condition on blockquote : we mustn't add quotes as a paragraph ! It's important since some wikipedia pages have an intro in which there is a quote.
                    while tag.name == 'p':                    
                        plot+=tag.text
                        tag = tag.find_next_sibling()


                else:
                    while tag.text == "\n" or tag.find_parent('table') != None or tag.find_parent('blockquote') != None:
                        tag = tag.find_next("p")


                    while tag.name == 'p':
                        plot+=tag.text
                        tag = tag.find_next_sibling()
            else:
                pass

        else :
            pass


        if intro != "":
            intro = intro.replace("\n", "")
            tot.append(intro)
        else :
            pass

        if plot != "":
            plot = plot.replace("\n", "")
            tot.append(plot)
        else :
            pass


        nametsv = "textsfolder/" + link_movie[:-5] + ".txt"
        with open(nametsv, 'w') as f_output:   
            f_output.write("\n".join(tot))

            
    except Exception:
        pass

## Creating the encoding (with transformers) and storing them

In [3]:
def store_data(name_output = "encoding.ann"):
    
    model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

    f = 768
    t = AnnoyIndex(f, 'angular')  # Length of item vector that will be indexed
    
    with open("idsmovies/ids.txt", 'w') as id_and_names :
    
        for i,valeur in enumerate(os.listdir("textsfolder")):
            all_text = []
            with open("textsfolder/" + valeur, 'r') as f:   
                for line in f:
                    all_text.append(line)

            try:
                
                v = model.encode(" ".join(all_text[1:]))
                t.add_item(i, v)

                id_and_names.write(str(i) + " " + all_text[0])
                
            except:
                
                print(valeur)

    t.build(10) # 10 trees
    t.save(name_output)

    print("Done")

In [4]:
def load_corpus(file_name = 'encoding.ann'):
    f = 768
    u = AnnoyIndex(f, 'angular')
    u.load(file_name)
    return u

def import_enc(file_name = "encoding.ann"):
    model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
    embeddings = load_corpus(file_name)
    return(model, embeddings)

def produce_prediction(query_text, model, embeddings, top_n = 3):
    query = model.encode(query_text)
    nearest = embeddings.get_nns_by_vector(query, top_n, include_distances = True)
    return nearest

## Storing the embeddings

In [5]:
store_data()

10_(film).txt
American_Graffiti.txt
A_Bridge_Too_Far_(film).txt
A_Matter_of_Time_(1976_film).txt
Being_There.txt
Bridge_to_the_Sun.txt
Charly.txt
Chetniks!_The_Fighting_Guerrillas.txt
Conquest_(1937_film).txt
Dark_Star_(film).txt
Five_Graves_to_Cairo.txt
Gigi_(1958_film).txt
Godzilla,_King_of_the_Monsters!.txt
Green_Fields_(film).txt
Hatari!.txt
Hawaii_(1966_film).txt
Head_(film).txt
Hell_in_the_Pacific.txt
He_Who_Gets_Slapped.txt
Honolulu_(film).txt
I%27ll_Never_Heil_Again.txt
Irma_la_Douce.txt
Island_in_the_Sky_(1953_film).txt
King_Kong_Escapes.txt
Kissin%27_Cousins.txt
Lost_Command.txt
Love_At_First_Bite.txt
Men_in_White_(1934_film).txt
Midnight_Express_(film).txt
Mr._B_Natural.txt
Mr._Imperium.txt
Mutiny_on_the_Bounty_(1962_film).txt
None_But_The_Brave.txt
One,_Two,_Three.txt
One_Froggy_Evening.txt
Party_at_Kitty_and_Stud%27s.txt
Phaedra_(film).txt
Public_Hero_No._1.txt
Raid_on_Entebbe_(film).txt
Red_Sun.txt
Retreat,_Hell!.txt
Rio_Bravo_(film).txt
Sybil_(1976_film).txt
Tabu_(1931_f

In [6]:
model, embeddings = import_enc()

In [27]:
dict_ids_names = {}
with open("idsmovies/ids.txt") as ids_movies:
    for line in ids_movies:
        split_line = line.strip("\n").split(" ")
        dict_ids_names[int(split_line[0])] = " ".join(split_line[1:])

In [52]:
pred = produce_prediction("The Godfather is a film produced by Francis Ford Coppola and which was created in 1972 ", model, embeddings)

In [53]:
for prediction in pred[0] :
    print(dict_ids_names[prediction])

The Godfather
The Godfather Part II
Crazy Joe


In [57]:
pred[0]

[7280, 7281, 1632]

In [60]:
embeddings.get_nns_by_item(7281, 2)

[7281, 7280]

This is logical, since The Godfather 1 is close to The Godfather 2. 

It seems that the model works well.